In [4]:
import csv, json, os, pandas, re, scipy, scipy.sparse, shutil
import subprocess, sys, threading, time, urllib2

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec(open('tileserve.py').read())


set default_psql_database to census2010
gcc -pthread -shared -rdynamic -fno-strict-aliasing -g -DNDEBUG -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/rsargent/anaconda2/include/python2.7 -L/home/rsargent/anaconda2/lib/python2.7/site-packages -lpython2.7 /tmp/tmpHtjm6N.c -o /tmp/tmp1JiYwu-000000.so
404
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>



In [38]:
# Removing the least recent takes O(N) time;  could be make more efficient if needed for larger dicts

class LruDict:
    def __init__(self, max_entries):
        self.max_entries = max_entries
        self.entries = {}
        self.usecount = 0
    
    def has(self, key):
        return key in self.entries
    
    def get(self, key):
        print 'getting %s' % key
        self.use(key)
        return self.entries[key]['data']
    
    def use(self, key):
        self.usecount += 1
        self.entries[key]['lastuse'] = self.usecount

    def insert(self, key, val):
        print 'inserting %s' % key
        self.entries[key] = {'data':val}
        self.use(key)
        if len(self.entries) > self.max_entries:
            lru_key, lru_val = None, None
            for key, val in self.entries.iteritems():
                if not lru_val or val['lastuse'] < lru_val['lastuse']:
                    lru_key, lru_val = key, val
            if lru_val:
                del self.entries[lru_key]

In [46]:
column_cache = LruDict(200) # max entries

def map_as_array(path):
    return numpy.memmap(path, dtype=numpy.float32, mode='r')

def load_column(dataset, column):
    cache_key = '{dataset}.{column}'.format(**locals())
    if column_cache.has(cache_key):
        return column_cache.get(cache_key)
    dir = '{cache_dir}/{dataset}'.format(cache_dir=cache_dir, **locals())
    if not os.path.exists(dir):
        msg = 'Dataset named "{dataset}" not found.<br><br><a href="{dataroot}">List valid datasets</a>'.format(dataroot=dataroot(), **locals())
        raise InvalidUsage(msg)
    cache_filename_prefix = dir + '/' + column
    if os.path.exists(cache_filename_prefix + '.float32'):
        data = map_as_array(cache_filename_prefix + '.float32')
    elif os.path.exists(cache_filename_prefix + '.numpy'):
        data = numpy.load(open(cache_filename_prefix + '.numpy'))
    else:
        msg = 'Column named "{column}" in dataset "{dataset}" not found.<br><br><a href="{dataroot}/{dataset}">List valid columns from {dataset}</a>'.format(dataroot=dataroot(), **locals())
        raise InvalidUsage(msg)

    column_cache.insert(cache_key, data)
    return data

testnow = False

if testnow:
    print load_column('census1990_block2010', 'P0010001').sum()
    print load_column('acs2015_5year_tract2010', 'B01001_001').sum()
    print load_column('census1990_block2010', 'P0010001').sum()
    print load_column('acs2015_5year_tract2010', 'B01001_001').sum()
    print load_column('census1990_block2010', 'P0010001').sum()
    print load_column('acs2015_5year_tract2010', 'B01001_001').sum()


In [52]:
expression_cache = LruDict(100) # 

def eval_layer_column(expr):
    try:
        return eval_(ast.parse(expr, mode='eval').body)
    except SyntaxError,e:
        raise InvalidUsage('<pre>' + traceback.format_exc(0) + '</pre>')

print eval_layer_column('acs2015_5year_tract2010.B01001_001 - acs2015_5year_tract2010.B01001_001').sum()
eval_layer_column('census1990_block2010.P0010001 - acs2015_5year_tract2010.B01001_001')


getting acs2015_5year_tract2010.B01001_001
getting acs2015_5year_tract2010.B01001_001
0.0
getting census1990_block2010.P0010001
getting acs2015_5year_tract2010.B01001_001


memmap([  0.        ,  81.        ,   0.        , ...,   1.77455461,
         0.        ,   0.        ], dtype=float32)

In [33]:
column_cache.entries

{'acs2015_5year_tract2010.B01001_001': {'data': memmap([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32),
  'lastuse': 2}}